In [1]:
using TimeSeries
using Pkg
using DataFrames,DataFramesMeta
using Dates
using Statistics
using Flux
using Flux: @epochs
using Plots
using Hyperopt
using HTTP
using Sockets
using JSON


In [2]:
include("env.jl")

"VVp9vMTjNvSEUqJJuqpRLw5r4oUq2uFGbX2VipCE"

In [3]:
using Distributed

In [4]:
addprocs(4)
nworkers()

4

In [5]:
ENV["PYTHON"] = "/Users/kahingleung/PycharmProjects/mylightning/venv/bin/python3.8"
Pkg.build("PyCall")

   Building Conda ─→ `~/.julia/packages/Conda/x5ml4/deps/build.log`
   Building PyCall → `~/.julia/packages/PyCall/tqyST/deps/build.log`


In [6]:
using PyCall
function getStock(sym,period)
    yf = pyimport("yfinance")
    #sym = "gbtc"
    #sym = "2840.HK"
    ticker = yf.Ticker(sym)
    etf = ticker.history(period=period)

    function pd_to_df(df_pd)
        df= DataFrame()
        for col in df_pd.columns
            df[!, col] = getproperty(df_pd, col).values
        end
        df[!,:Date] = collect(df_pd[:index])
        return df
    end
    etf = pd_to_df(etf)

    transform!(etf, :Date => ( x -> Date.(Dates.year.(x),Dates.month.(x),Dates.day.(x)) ) => :ondate)
    for c in ["High","Low","Open","Close","Stock Splits"]
        transform!(etf,c => (x -> Float32.(x) )=> c)
    end
    return select(etf,[:Open,:Close,:High,:Low,:Volume,:ondate])
end

getStock (generic function with 1 method)

In [28]:
function wgetStock(sym::String,period::Int)
    include("env.jl")
    fromYear = Dates.format(today()-Year(period), "yyyy-mm-dd")
    headers = ["APCA-API-KEY-ID" => ENV["APCA-API-KEY-ID"], "APCA-API-SECRET-KEY" => ENV["APCA-API-SECRET-KEY"] ]
    url_query = "https://data.alpaca.markets/v1/bars/day?symbols="*sym*"&after="*fromYear*"T00:00:00&limit=1000"
    raw_response = HTTP.request("GET", url_query, headers)
    rsp = JSON.parse(String(raw_response.body))
    df2 = DataFrame(t = Int[],h = Float32[],l = Float32[],o = Float32[],c = Float32[],v = Float32[] )
    for (sym,v) in rsp
        for q in v
            push!(df2,q)
        end
    end
    transform!(df2, :t => ( x -> Dates.unix2datetime.(x) ) => :ondate)
    DataFrames.rename!(df2,:h => :High)
    DataFrames.rename!(df2,:l => :Low)
    DataFrames.rename!(df2,:o => :Open)
    DataFrames.rename!(df2,:c => :Close)
    DataFrames.rename!(df2,:v => :Volume)
    sort!(df2,order(:ondate))
    return select(df2,[:Open,:Close,:High,:Low,:Volume,:ondate])
end

wgetStock (generic function with 1 method)

In [8]:
function toPL(PL_ts)
    function f_PL(values)
        open = values[1]
        close = values[2]
        if close - open > 0
            pl = 1
        else
            pl = 0
        end
        [pl,0]
    end     
    PL = TimeSeries.rename(TimeSeries.map((timestamp, values) -> (timestamp, f_PL(values)),PL_ts)[:Open],Symbol("PL") )
    PL = TimeSeries.map((timestamp, values) -> (timestamp, Int(values)),PL)
end

toPL (generic function with 1 method)

In [9]:
function toRSI(price_ts,loopback)
    pct = percentchange(price_ts)
    upidx = findall(pct["Close"] .> 0)
    downidx = findall(pct["Close"] .< 0)
    up = map( (timestamp, values) -> (timestamp, if values < 0 ; 0 ; else ;values ;end), pct)
    down = map( (timestamp, values) -> (timestamp, if values < 0 ; abs(values) ; else ;0 ;end), pct)
    up_roll = moving(mean, up, loopback)
    down_roll = moving(mean, down, loopback)
    function f_rsi(values)
        x = values[1]
        y = values[2]
        rsi = 100 - (100 / (1 + x/y) )
        [rsi,0]
    end
    updown = TimeSeries.rename(TimeSeries.merge(up_roll,down_roll), [:up,:down])
    rsi_ts = TimeSeries.rename(TimeSeries.map((timestamp, values) -> (timestamp, f_rsi(values)), updown)[:up],Symbol("RSI-",loopback))
    return rsi_ts
end

toRSI (generic function with 1 method)

In [10]:
function toORSI(price_ts,day0,day1)
    #pct = percentchange(price_ts)
    result_rsi = price_ts
    result_rsima = price_ts
    result_orsi = price_ts
    for i in day0:day1
        rsi_ts = toRSI(price_ts,i)
        result_rsi = merge(result_rsi,rsi_ts,method=:inner)
        for j in day0:day1
            rsi_ma = TimeSeries.rename(moving(mean,rsi_ts,j),Symbol("RSIMA-",i,"-",j))
            orsi = TimeSeries.rename(rsi_ts .- rsi_ma , Symbol("ORSI-",i,"-",j) )
            result_orsi = merge(result_orsi,orsi,method=:inner)
        end
    end
    #println(colnames(result_orsi))
    return result_orsi
end

toORSI (generic function with 1 method)

In [11]:
function toMatrixORSI(data,featureIdx,targetIdx)
    println(colnames(data)[1:5])
    #featureIdx=3
    data = values(data)
    X = data[:,featureIdx:end]
    #b = maximum(X)
    #a = minimum(X)
    #println(a,"-",b)
    #X = ( b .- X ) ./ (b-a+1)
    print(size(X))
    N = size(X)[1]
    k = Int(sqrt(size(X)[2]))
    println(k)
    println(X[1,1:4])
    println(X[2,1:4])
    println(X[2,k+1:k+4])
    X = transpose(X)
    X = reshape(X,(k,k,N))
    println(X[1:4,1,1])
    println(X[1:4,1,2])
    println(X[1:4,2,2])
    #targetIdx=1
    Y = data[:,targetIdx]
    return (X,Y)
end

toMatrixORSI (generic function with 1 method)

In [12]:
function toReturn(price_ts)
    pct = TimeSeries.rename(percentchange(price_ts),[:return])
    pct = moving(mean,pct,5)
    return TimeSeries.map((timestamp, values) -> (timestamp, 100*values), pct)
end   

toReturn (generic function with 1 method)

In [13]:
function preprocessing(etf,day0,day1,mlclass)
    PL = select(etf,[:Open,:Close,:ondate])
    price = select(etf,[:Close,:ondate])
    PL_ts = TimeArray(PL,timestamp=:ondate)
    PL_ts = toPL(PL_ts)
    pct = TimeSeries.rename(percentchange(TimeArray(price,timestamp=:ondate)),[:return])
    price_ts = TimeArray(price,timestamp=:ondate)
    pct = toReturn(price_ts)
    #day0=5
    #day1=30
    result_orsi = toORSI(price_ts,day0,day1)
    tgt = merge(PL_ts,pct,method=:inner)
    data = merge(tgt,result_orsi,method=:inner)
    featureIdx = 4
    if mlclass == :reg
        tgt = 2
    else
        tgt = 1
    end
        
    (X,Y) = toMatrixORSI(data,featureIdx,tgt)
end

preprocessing (generic function with 1 method)

In [14]:
function timeseriesDataset(X,Y,seqlen,mlclass)
    xtrain = Array{Float32,3}[]
    ytrain = Array{Float32,1}[]
    len = length(Y)
    for i in 1:len-seqlen-1
        mx = X[:,:,i:i+seqlen-1]
        xtrain = vcat(xtrain,[mx])
        if mlclass == :reg
            my = Y[i+seqlen]
        else
            my = Flux.onehot(Y[i+seqlen], [0,1])
        end
        ytrain = vcat(ytrain,[my])
    end
    xcurrent = [X[:,:,end-seqlen+1:end]]
    return (xtrain,ytrain,xcurrent)
end
    

timeseriesDataset (generic function with 1 method)

In [15]:
function build_model(Nh,seqlen, delta)
    a = floor(Int8,Nh)
    sz = ceil(Int8,delta/8)
    
    println(delta,"-",sz)
    return Chain(
    x -> Flux.unsqueeze(x,4),
    # First convolution, operating upon a 28x28 image
    Conv((2, 2), seqlen=>a, pad=(1,1), relu),
    MaxPool((2,2)),

    # Second convolution, operating upon a 14x14 image
    Conv((2, 2), a=>Nh, pad=(1,1), relu),
    MaxPool((2,2)),

    # Third convolution, operating upon a 7x7 image
    Conv((2, 2), Nh=>Nh, pad=(1,1), relu),
    MaxPool((2,2)),

    # Reshape 3d tensor into a 2d one using `Flux.flatten`, at this point it should be (3, 3, 32, N)
    Flux.flatten,
    #Dropout(0.1),
    Dense(sz*sz*Nh, 2),
    softmax
    )
end

build_model (generic function with 1 method)

In [16]:
function build_reg_model(Nh,seqlen, delta)
    a = floor(Int8,Nh)
    sz = ceil(Int8,delta/8)
    
    println(delta,"-",sz)
    return Chain(
    x -> Flux.unsqueeze(x,4),
    # First convolution, operating upon a 28x28 image
    Conv((2, 2), seqlen=>a, pad=(1,1), relu),
    MaxPool((2,2)),

    # Second convolution, operating upon a 14x14 image
    Conv((2, 2), a=>Nh, pad=(1,1), relu),
    MaxPool((2,2)),

    # Third convolution, operating upon a 7x7 image
    Conv((2, 2), Nh=>Nh, pad=(1,1), relu),
    MaxPool((2,2)),

    # Reshape 3d tensor into a 2d one using `Flux.flatten`, at this point it should be (3, 3, 32, N)
    Flux.flatten,
    Dropout(0.1),
    (x->transpose(x)),
    GRU(1,Nh),
    GRU(Nh,Nh),
    (x -> x[:,end]),
    Dense(Nh, 1),
    (x -> x[1])
    )
end

build_reg_model (generic function with 1 method)

In [17]:
function accuracy(m,xtest,ytest)
    prediction = m.(xtest)
    len = length(ytest)
    m = zeros(2,2)
    for i in 1:len
        yh = Flux.onecold(prediction[i],[0,1])
        y = Flux.onecold(ytest[i],[0,1])
        a = Int(yh[1])+1
        b = y+1
        m[a,b] += 1
    end
    m = m ./ sum(m)
    return m
end

accuracy (generic function with 1 method)

In [18]:
function reg_accuracy(m,xtest,ytest)
    prediction = m.(xtest)
    len = length(ytest)
    m = zeros(2,2)
    for i in 1:len
        yh = prediction[i] > 0 ? 1 : 0
        y = ytest[i] > 0 ? 1 : 0
        a = Int(yh[1])+1
        b = y+1
        m[a,b] += 1
    end
    m = m ./ sum(m)
    return m
end

reg_accuracy (generic function with 1 method)

In [19]:
function plot_eval(m,xtest,ytest)
    prediction = m.(xtest)
    len = length(ytest)
    #println(len)
    pred = Float32[]
    actual = Float32[]
    total = 0
    for i in 1:len
        yh = Flux.onecold(prediction[i],[0,1])
        y = Flux.onecold(ytest[i],[0,1])
        append!(pred,yh)
        append!(actual,y)
    end
    plot(1:len,pred,size = (1000, 700),color="red",tickfontsize=28,guidefontsize=28,legendfontsize=28)
    plot!(1:len,actual,size = (1000, 700),color="blue",tickfontsize=28,guidefontsize=28,legendfontsize=28)
end

plot_eval (generic function with 1 method)

In [20]:
function myTrain(etf,seqlen,Nh,lr,mm,day0,day1, mlclass,i)
    #seqlen=7
    #mlclass = :clf
    (X,Y) = preprocessing(etf,day0,day1,mlclass)
    (xtrain,ytrain,xcurrent) = timeseriesDataset(X,Y,seqlen,mlclass)
    thd = 150
    xtest,ytest = xtrain[end-thd+1:end], ytrain[end-thd+1:end]
    xtrain,ytrain = xtrain[1:end-thd], ytrain[1:end-thd]
    batchsize = 20
    train_loader = Flux.Data.DataLoader(xtrain,ytrain, batchsize=batchsize,shuffle=false)
    #Nh=10
    delta = day1 - day0 + 1
    if mlclass == :reg
        m = build_reg_model(Nh,seqlen,delta)
    else
        m = build_model(Nh,seqlen,delta)
    end
    function loss(x, y)    
        yh = m.(x)
        #println(yh)
        #println(y)
        #println(Flux.Losses.logitcrossentropy(yh[1], y[1]))
        return sum(Flux.Losses.logitcrossentropy.(yh, y))
    end
    function acc_loss(x,y)
        (acc,y0,y1,y2) = accuracy(m,x,y)
        return (1-acc-y0+y1)*abs(y2-0.5)
    end
    function mse_loss(x,y)
        yh = m.(x)
        e = Flux.mae(yh,y)
        return e
    end
    evalcb() = @show loss(xtest,ytest)
    num_epoch = 25
    #lr = 0.01
    if mlclass == :reg
        @epochs num_epoch Flux.train!(mse_loss,Flux.params(m),train_loader,RMSProp(lr,mm))
        confmx = reg_accuracy(m,xtest,ytest)
        prediction = m.(xtest)
        println(size(prediction))
        mseloss = mse_loss(xtest,ytest)
        println("mse=",mseloss)
        objcost = mseloss-1*(confmx[1,1]+confmx[2,2]-confmx[1,2]-confmx[2,1])
        #p = map(x -> x[end],prediction)
        #y = map(x -> x[end],ytest)
        #lp = getindex(size(prediction),1)
        #ly = getindex(size(y),1)
        plot(1:length(prediction),prediction,size = (1000, 700),color="red",tickfontsize=28,guidefontsize=28,legendfontsize=28)
        plot!(1:length(ytest),ytest,size = (1000, 700),color="blue",tickfontsize=28,guidefontsize=28,legendfontsize=28)
        dir="/Users/kahingleung/Downloads/gold-julia/"
        png("/Users/kahingleung/Downloads/gold-julia/plot-"*string(i)*".png")
        println("current size:", size(xcurrent[1]))
        future = m.(xcurrent)
        future = future[1]
        println("future:",future)
    else
        @epochs num_epoch Flux.train!(loss,Flux.params(m),train_loader,RMSProp(lr,mm))
        confmx = accuracy(m,xtest,ytest)
        objcost = -1*(confmx[1,1]+confmx[2,2]-confmx[1,2]-confmx[2,1])
        println("current size:", size(xcurrent[1]))
        future = m.(xcurrent)
        future = future[1]
        println("future:",future)
    end
    println("accuracy:",(confmx[1,1]+confmx[2,2]))
    println(confmx[1,:])
    println(confmx[2,:])
    
    return (m,objcost,xtest,ytest,confmx,future)
end
function myObjective(etf,seqlen,Nh,lr,mm,day0,day1,mlclass,i)
    (m,objcost,xtest,ytest,confmx,future) = myTrain(etf,seqlen,Nh,lr,mm,day0,day1,mlclass,i)
    Flux.reset!(m)
    return objcost
end

myObjective (generic function with 1 method)

In [21]:
#sym="2840.HK"
sym="GLD"
etf = wgetStock(sym,3)

,Open,Close,High,Low,Volume,ondate
,Float32,Float32,Float32,Float32,Float32,DateTime
1,116.26,115.83,116.57,115.625,5.55043e6,2017-02-02T05:00:00
2,115.73,116.14,116.365,115.71,7.0282e6,2017-02-03T05:00:00
3,117.07,117.68,117.74,116.74,7.43806e6,2017-02-06T05:00:00
4,117.31,117.46,117.74,117.2,7.88362e6,2017-02-07T05:00:00
5,118.09,118.17,118.59,117.81,8.00184e6,2017-02-08T05:00:00
6,118.32,117.31,118.58,117.2,7.4162e6,2017-02-09T05:00:00
7,116.68,117.59,117.86,116.67,7.19955e6,2017-02-10T05:00:00
8,116.73,116.79,116.95,116.15,5.89761e6,2017-02-13T05:00:00
9,117.52,116.93,117.56,116.38,5.92856e6,2017-02-14T05:00:00


In [30]:
@everywhere include("installPkg.jl")

   Updating registry at `~/.julia/registries/General`


      From worker 4:	   Updating registry at `~/.julia/registries/General`
      From worker 5:	   Updating registry at `~/.julia/registries/General`
      From worker 3:	   Updating registry at `~/.julia/registries/General`
      From worker 2:	   Updating registry at `~/.julia/registries/General`


######################################################################### 100.0%                                 21.1%


######################################################################### 100.0%                    
######################################################################### 100.0%                    
######################################################################### 100.0%                    


No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving packa

...

┌ Warning: The call to compilecache failed to create a usable precompiled cache file for OsRSIConv [96985222-94fb-4572-aef8-be4c18e2ae3a]
│   exception = ErrorException("Required dependency TradeAPI [c0095e52-a479-476f-bc54-3cccc04d9c53] failed to load from a cache file.")
└ @ Base loading.jl:1042
┌ Warning: Package OsRSIConv does not have Pkg in its dependencies:
│ - If you have OsRSIConv checked out for development and have
│   added Pkg as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with OsRSIConv
│ Loading Pkg into OsRSIConv from project dependency, future warnings for OsRSIConv are suppressed.
└ @ nothing nothing:910
   Updating registry at `~/.julia/registries/General`
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to

      From worker 5:	┌ Warning: Package OsRSIConv does not have Pkg in its dependencies:
      From worker 5:	│ - If you have OsRSIConv checked out for development and have
      From worker 5:	│   added Pkg as a dependency but haven't updated your primary
      From worker 5:	│   environment's manifest file, try `Pkg.resolve()`.
      From worker 5:	│ - Otherwise you may need to report an issue with OsRSIConv
      From worker 5:	└ Loading Pkg into OsRSIConv from project dependency, future warnings for OsRSIConv are suppressed.
      From worker 5:	   Updating registry at `~/.julia/registries/General`
      From worker 5:	No Changes to `~/.julia/environments/v1.5/Project.toml`
      From worker 5:	No Changes to `~/.julia/environments/v1.5/Manifest.toml`
      From worker 5:	  Resolving package versions...
      From worker 5:	No Changes to `~/.julia/environments/v1.5/Project.toml`
      From worker 5:	No Changes to `~/.julia/environments/v1.5/Manifest.toml`
      From worker 5:	  Reso

┌ Warning: The call to compilecache failed to create a usable precompiled cache file for OsRSIConv [96985222-94fb-4572-aef8-be4c18e2ae3a]
│   exception = Required dependency TradeAPI [c0095e52-a479-476f-bc54-3cccc04d9c53] failed to load from a cache file.
└ @ Base loading.jl:1042
┌ Warning: Package OsRSIConv does not have Pkg in its dependencies:
│ - If you have OsRSIConv checked out for development and have
│   added Pkg as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with OsRSIConv
└ Loading Pkg into OsRSIConv from project dependency, future warnings for OsRSIConv are suppressed.


      From worker 5:	   Updating registry at `~/.julia/registries/General`
      From worker 5:	No Changes to `~/.julia/environments/v1.5/Project.toml`
      From worker 5:	No Changes to `~/.julia/environments/v1.5/Manifest.toml`
      From worker 5:	  Resolving package versions...
      From worker 5:	No Changes to `~/.julia/environments/v1.5/Project.toml`
      From worker 5:	No Changes to `~/.julia/environments/v1.5/Manifest.toml`
      From worker 5:	  Resolving package versions...
      From worker 5:	No Changes to `~/.julia/environments/v1.5/Project.toml`
      From worker 5:	No Changes to `~/.julia/environments/v1.5/Manifest.toml`
      From worker 5:	  Resolving package versions...
      From worker 5:	No Changes to `~/.julia/environments/v1.5/Project.toml`
      From worker 5:	No Changes to `~/.julia/environments/v1.5/Manifest.toml`
      From worker 5:	  Resolving package versions...
      From worker 5:	No Changes to `~/.julia/environments/v1.5/Project.toml`
      From worke

LoadError: On worker 2:
LoadError: IOError: unlink: no such file or directory (ENOENT)
uv_error at ./libuv.jl:97 [inlined]
unlink at ./file.jl:918
#rm#9 at ./file.jl:268
#rm#9 at ./file.jl:278 (repeats 3 times)
#checkfor_mv_cp_cptree#10 at ./file.jl:306
#cp#12 at ./file.jl:345
#107 at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.5/Pkg/src/Types.jl:1134
#mktempdir#20 at ./file.jl:709
mktempdir at ./file.jl:707 (repeats 2 times)
#update_registries#105 at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.5/Pkg/src/Types.jl:1119
#up#106 at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.5/Pkg/src/API.jl:227
up at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.5/Pkg/src/API.jl:221 [inlined]
#up#37 at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.5/Pkg/src/API.jl:67 [inlined]
up at /Users/julia/buildbot/worker/package_macos64/build/usr/share/jul

In [29]:
tstart = time()
ho = @phyperopt for i=20,
            sampler = RandomSampler(), # This is default if none provided
            seqlen = StepRange(3, 5, 20),
            Nh = StepRange(5,3, 20),
            delta = StepRange(10,5, 25),
            #lr =  LinRange(1e-5,9e-5,10),
            lr =  exp10.(LinRange(-4,-3,10)),
            mm =  LinRange(0.75,0.95,5),
            day0 = StepRange(5,3, 10)
    println(i,"-",seqlen,"-",Nh,"-",lr,"-",mm,"-",day0,"-",delta)
    sym="GLD"
    etf = wgetStock(sym,3)
   @show myObjective(etf,seqlen,Nh,lr,mm,day0,day0+delta,:reg,i)
end
tend=time()
et=tend-tstart
@show et

LoadError: On worker 2:
KeyError: key Hyperopt [93e5fe13-2215-51db-baaf-2e9a34fb2712] not found
getindex at ./dict.jl:467 [inlined]
root_module at ./loading.jl:968 [inlined]
deserialize_module at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.5/Serialization/src/Serialization.jl:953
handle_deserialize at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.5/Serialization/src/Serialization.jl:855
deserialize at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.5/Serialization/src/Serialization.jl:773
handle_deserialize at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.5/Serialization/src/Serialization.jl:863
deserialize at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.5/Serialization/src/Serialization.jl:773
deserialize_expr at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.5/Serialization/src/Serialization.jl:1163
h

In [23]:
best_params, min_f = ho.minimizer, ho.minimum

LoadError: UndefVarError: ho not defined

In [24]:
seqlen=best_params[1]
Nh=best_params[2]
delta=best_params[3]
lr=best_params[4]
mm=best_params[5]
day0=best_params[6]
(m,acc,xtest,ytest,confmx,future) = myTrain(etf,seqlen,Nh,lr,mm,day0,day0+delta,:reg,21)

LoadError: UndefVarError: best_params not defined

In [25]:
#20-13-20-0.000774263682681127-0.8-5-15
seqlen=13
Nh=20
delta=15
lr=0.000774
mm=0.8
day0=5
println(seqlen,"-",Nh,"-",lr,"-",mm,"-",day0,"-",delta)
df = DataFrame(symbol = String[], accuracy = Float32[], future = Float32[])
#list = ["FB","AAPL","AMZN","GOOG","NFLX","SQ","MTCH","AYX","ROKU","TTD"]
#sym="GLD"
filepath="list.txt"
list=readlines(filepath)
println(list)
for sym in list
    etf = wgetStock(sym,3)
    (m,cost,xtest,ytest,confmx,future) = myTrain(etf,seqlen,Nh,lr,mm,day0,day0+delta,:reg,sym)
    acc = (confmx[1,1]+confmx[2,2])
    push!(df,[sym,acc,future])
end
df

13-20-0.000774-0.8-5-15
["FB", "AAPL", "AMZN", "GOOG", "GLD", "SQ", "MTCH", "AYX", "ROKU", "TTD"]
[:PL, :return, :Close, Symbol("ORSI-5-5"), Symbol("ORSI-5-6")]
(961, 256)16
Float32[27.271889, 30.98352, 35.69597, 35.564808]
Float32[10.94191, 17.212006, 20.885178, 25.44368]
Float32[16.267258, 20.654953, 22.933235, 27.553497]
Float32[27.271889, 30.98352, 35.69597, 35.564808]
Float32[10.94191, 17.212006, 20.885178, 25.44368]
Float32[16.267258, 20.654953, 22.933235, 27.553497]
16-2


┌ Info: Epoch 1
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 2
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 3
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 4
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 5
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 6
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 7
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 8
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 9
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 10
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 11
└ @ Main /Us

(150,)
mse=0.5814315
current size:(16, 16, 13)
future:1.0594395
accuracy:0.7666666666666666
[0.29333333333333333, 0.03333333333333333]
[0.2, 0.47333333333333333]
[:PL, :return, :Close, Symbol("ORSI-5-5"), Symbol("ORSI-5-6")]
(961, 256)16
Float32[16.231041, 19.343964, 20.637276, 21.698761]
Float32[10.947784, 14.344818, 17.422882, 18.917511]
Float32[7.9365463, 11.998787, 14.303268, 14.393501]
Float32[16.231041, 19.343964, 20.637276, 21.698761]
Float32[10.947784, 14.344818, 17.422882, 18.917511]
Float32[7.9365463, 11.998787, 14.303268, 14.393501]
16-2


┌ Info: Epoch 1
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 2
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 3
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 4
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 5
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 6
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 7
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 8
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 9
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 10
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 11
└ @ Main /Us

(150,)
mse=1.263172
current size:(16, 16, 13)
future:0.415381
accuracy:0.5933333333333334
[0.0, 0.0]
[0.4066666666666667, 0.5933333333333334]
[:PL, :return, :Close, Symbol("ORSI-5-5"), Symbol("ORSI-5-6")]
(961, 256)16
Float32[26.589058, 31.39251, 34.526936, 38.379845]
Float32[20.043045, 27.163559, 32.0569, 35.467377]
Float32[21.411003, 27.780403, 31.748081, 34.407066]
Float32[26.589058, 31.39251, 34.526936, 38.379845]
Float32[20.043045, 27.163559, 32.0569, 35.467377]
Float32[21.411003, 27.780403, 31.748081, 34.407066]
16-2


...

┌ Info: Epoch 1
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 2
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 3
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 4
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 5
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 6
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 7
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 8
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 9
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 10
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 11
└ @ Main /Us

(150,)
mse=0.88299966
current size:(16, 16, 13)
future:0.9264187
accuracy:0.7866666666666667
[0.19333333333333333, 0.12]
[0.09333333333333334, 0.5933333333333334]
[:PL, :return, :Close, Symbol("ORSI-5-5"), Symbol("ORSI-5-6")]
(961, 256)16
Float32[9.057148, 12.1297035, 14.0975685, 17.439037]
Float32[14.679604, 19.428112, 22.616821, 24.809887]
Float32[9.660088, 13.9197235, 17.061794, 18.130684]
Float32[9.057148, 12.1297035, 14.0975685, 17.439037]
Float32[14.679604, 19.428112, 22.616821, 24.809887]
Float32[9.660088, 13.9197235, 17.061794, 18.130684]
16-2


┌ Info: Epoch 1
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 2
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 3
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 4
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 5
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 6
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 7
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 8
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 9
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 10
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 11
└ @ Main /Us

(150,)
mse=1.2657808
current size:(16, 16, 13)
future:0.41476572
accuracy:0.6666666666666666
[0.0, 0.0]
[0.3333333333333333, 0.6666666666666666]


,symbol,accuracy,future
,String,Float32,Float32
1,FB,0.766667,1.05944
2,AAPL,0.593333,0.415381
3,AMZN,0.686667,0.846173
4,GOOG,0.82,0.706086
5,GLD,0.813333,-0.0200774
6,SQ,0.8,-0.122053
7,MTCH,0.606667,0.191848
8,AYX,0.666667,1.12813
9,ROKU,0.786667,0.926419


In [26]:
df

,symbol,accuracy,future
,String,Float32,Float32
1,FB,0.766667,1.05944
2,AAPL,0.593333,0.415381
3,AMZN,0.686667,0.846173
4,GOOG,0.82,0.706086
5,GLD,0.813333,-0.0200774
6,SQ,0.8,-0.122053
7,MTCH,0.606667,0.191848
8,AYX,0.666667,1.12813
9,ROKU,0.786667,0.926419


In [27]:
#using Pkg; Pkg.generate("OsRSIConv")